# LIME - Local Interpretable Model-Agnostic Explanations

This assignment uses LIME, a package that provides "Locally Interpetable Model-agnostic Explanations" for machine learning models.

- **Locally interpretable**: for each specific prediction, we can provide a relevant explanation.
- **Model-agnostic**: we can provide the same sort of explanation for models of different classes.

For more documentation of the LIME package, see [here](https://github.com/marcotcr/lime) and [here](https://lime-ml.readthedocs.io/en/latest/index.html). The original paper describing the method is [here](https://arxiv.org/pdf/1602.04938.pdf).

In [ ]:
!pip install lime # if you are running on JupyterLab, use !/opt/anaconda/bin/pip
!pip install xgboost

In [ ]:
%matplotlib inline

import sklearn.model_selection
import sklearn.metrics
import sklearn.datasets
import sklearn.ensemble
import sklearn.preprocessing
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import numpy as np
from IPython.display import Markdown, display
import matplotlib.pyplot as plt
import warnings
import lime
import lime.lime_tabular
from lime import submodular_pick
import xgboost
from xgboost import plot_importance
import json
from collections import OrderedDict

import pandas as pd

np.random.seed(1)

***
## Part 1: Prepare the data

We will be using the Adult dataset which we can use to predict whether a person makes over 50K dollars per year based on census information. A version of the dataset is found on UCI ML Repository : [here](https://archive.ics.uci.edu/ml/datasets/adult).

### Read in the data

In [ ]:
# Read in the data
colnames = ["Age", "Workclass", "fnlwgt", "Education", "Education-Num",
            "Marital-Status", "Occupation", "Relationship", "Race", "Sex",
            "Capital-Gain", "Capital-Loss","Hours-per-week", "Country",
            "income"]
data_df = pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data',
                      names = colnames)
print("Shape: ", data_df.shape)
data_df.head()

### Clean the data

In [ ]:
# TODO - Drop Education-Num, which has a functional dependency with Education


In [ ]:
# Get a list of feature names (excluding the outcome variable)
feature_names = data_df.columns[:-1]

In [ ]:
# Mark labels and encode them using sklearn
labels = data_df.iloc[:,-1]
le= sklearn.preprocessing.LabelEncoder()
le.fit(labels)
labels = le.transform(labels)
class_names = le.classes_
data = data_df.iloc[:,:-1]
le_label_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
print("Class names: ", class_names)
print("Label mapping: ", le_label_mapping)

In [ ]:
# Check if there are categorical variables that we need to make dummies for
print(data.dtypes)
# Get a list of which variables are categorical
categorical_features  = [i for i in range(len(data.dtypes)) if data.dtypes[i]=='object']
print("Indices of categorical features: ", categorical_features)

LIME will require us to provde categorical variables as a single column, not as dummies, so we can't just explode these columns the way we normally would during pre-processing.

Instead, we'll use some sklearn tools to take the following steps:
1. Encode the existing categories with a number corresponding to each category
2. Make a dictionary storing the relationship between the original string category and the number we've replaced it with (categorical_names)
3. Make a function that we can use down the line to transform categorical variables into dummies

In [ ]:
categorical_names = {}
for feature in categorical_features:
    print("Feature: ", feature)
    # Use label encoder to map categories to numbers
    le = sklearn.preprocessing.LabelEncoder()
    le.fit(data.iloc[:, feature])
    # Replace the categories with corresponding numbers in the original data
    data.iloc[:, feature] = le.transform(data.iloc[:, feature])
    # Store and print the mappings for reference later
    categorical_names[feature] = le.classes_
    print(categorical_names[feature])
    print("==================================================")

In [ ]:
# This variable is where we store the original names of each category for each variable
categorical_names

In [ ]:
# We can (and will) use this encoder function to transform the categorical columns into dummies--
# but we can't do that to the original dataset if we want to use LIME
encoder = ColumnTransformer(transformers=[('get_dummies', OneHotEncoder(), categorical_features)], remainder='passthrough')
encoder = encoder.fit(data)

### Split into training and test sets

We will us an 80/20 train-test split. We won't be doing any hyperparameter tuning during this lab, so no need to worry about a validation set.

In [ ]:
train, test, labels_train, labels_test = sklearn.model_selection.train_test_split(data, labels, train_size=0.80, random_state=10)
print("Train shape: ", train.shape)
print("Test shape: ", test.shape)


***
## Part 2: Train a model

We will be using gradient boosted decision trees as implemented by the [xgboost](https://github.com/dmlc/xgboost) package.

In [ ]:
# Fit the model
gbtree = xgboost.XGBClassifier(n_estimators=200, max_depth=5)
gbtree.fit(encoder.transform(train), labels_train)

In [ ]:
# Make predictions
pred_labels_test = gbtree.predict(encoder.transform(test))

# Calculate accuracy on the test set
print("Test set accuracy: ", sklearn.metrics.accuracy_score(labels_test, pred_labels_test))

***
## Part 3: Explaining predictions the usual way



### **Questions:**
*Hint:* Google is your best friend!
1. How would you normally explain the predictions of a single decision tree?

    **Write answer here:**


2. What is a gradient boosted decision tree?

    **Write answer here:**

3. Does that change for a gradient boosted decision tree?

    **Write answer here:**



We're going to be looking at a measure of feature importance. This is calculated for a single tree by counting how many splits occured on each variable. We arrive at a feature importance for the entire model by averaging the score for each feature across all trees in the forest.


In [ ]:
# Get feature names for the transformed dataset so that they'll show up on the plot -- don't worry about this bit
gbtree_features_orig = gbtree.get_booster().feature_names
gbtree_features = []
for cat_var in categorical_names:
    cat_var_names = [feature_names[cat_var] +" = " + cat for cat in categorical_names[cat_var]]
    gbtree_features.extend(cat_var_names)
gbtree_features.extend([feature_names[i] for i in range(len(feature_names)) if i not in categorical_features])
gbtree.get_booster().feature_names = gbtree_features

# Plot feature importances
plt.rcParams["figure.figsize"] = (10,20)
plot_importance(gbtree.get_booster())

Pick an interesting feature with a high importance in the plot above. Can you figure out the relationship of that feature with the outcome variable? (Is a higher or lower value more likely to indicate a high-income individual? For binary variables, is it 0 or 1?)

In [ ]:
# TODO - Your code here

The plot_importance function takes a parameter called 'importance_type', which you can read more about [in the documentation](https://xgboost.readthedocs.io/en/latest/python/python_api.html#module-xgboost.plotting). Try changing the importance type and re-making the plot. How does our explanation change?

In [ ]:
# TODO - Your code here

In [ ]:
# Replace the original feature names, which LIME will expect
gbtree.get_booster().feature_names = gbtree_features_orig

***
## Part 4: Explaining predictions with LIME

Time for LIME!

### LIME Tabular Explainer

The tabular explainer is how we get locally interpretable explanations for classification problems.

First, we initialize an explainer object that takes in all the information we stored/encoded earlier about the dataset.

In [ ]:
explainer = lime.lime_tabular.LimeTabularExplainer(train.values,
                                                   feature_names=feature_names,
                                                   class_names=class_names,
                                                   categorical_features=categorical_features,
                                                   categorical_names=categorical_names)

Next, we need to define a single function that takes in the form of data that LIME expects and returns the type of prediction that LIME expects. In particular,
- the input should be a numpy array (which we can get from a pandas df using .values)
- the input features should be human-understandable
- the input data should have each categorical variable in a single column
- the output should be a predicted probability (not a predicted class)

In [ ]:
predict_fn = lambda x: gbtree.predict_proba(encoder.transform(x)).astype(float)

Now we have everything we need to use the explainer. Let's get an explanation for one of the examples in the test set.

In [ ]:
i = 2
print('Actual class: ', labels_test[i])
# Get explanation
exp = explainer.explain_instance((test.values[i]), predict_fn, num_features=5)

In [ ]:
# Visualize the explanation
%matplotlib inline
fig = exp.as_pyplot_figure()

The explanations can also be exported as an html page (which we can render here in this notebook), using D3.js to render graphs. You could also save the html page to a file if you wanted to.

In [ ]:
exp.show_in_notebook(show_all=True)

The explanation can also be presented as a list of weighted features.

In [ ]:
exp.as_list()

### **Questions:**
4. How do we interpret this plot? That is, what do red and green mean? What's on the x axis?

    **Write answer here:**

5. How does this explanation differ from the feature importance explanation given above?

    **Write answer here:**


### **TODO - Generate LIME explanations (in whichever format you chose) for some other examples in the test set. Are similar features important? Add text + code cells below**

### LIME Submodular Picker

As we just saw, explanations can vary a lot depending on what instance we pick. While this is great for explaining a single prediction, it makes it hard to give someone general intuition for "how the model makes decisions." That's where the submodular picker comes in. It picks useful, representative examples that together give global explanation for the model.

In broad strokes, the algorithm does the following:
1. Calculate an explanation for all examples in the dataset
2. Determine which features are important in explaining a lot of predictions -- that is, features that seem globally important
3. Select (greedily) examples where the top globally important feature is part of the local explanation for that one example's prediction
3. Continue selecting examples until we've covered as many of the globally important features as possible, constrained by the number of features that the user wants returned (num_exps_desired)

You can read the details of how this is done in the paper.

In [ ]:
# Initialize the SP object
sp_obj = submodular_pick.SubmodularPick(explainer, train.values, predict_fn, sample_size=10,
                                        num_features=5, num_exps_desired=5)

The attribute V tells us the best indices from the test set to explain the overall predictions of the classifier.

In [ ]:
sp_obj.V

Now, we can get explantions for each of those examples:

In [ ]:
for ind in sp_obj.V:
    exp = explainer.explain_instance(test.values[ind], predict_fn, num_features=5)
    print("Actual class: ", labels_test[ind])
    exp.show_in_notebook(show_all=False)
    print("==========================")

### **Question:**

6. Based on these carefully chosen examples, what would you say to someone who wanted to know how our model makes decisions?

    **Write answer here:**
